In [2]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.optim as optim
import math
import random
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
from mpi4py import MPI 
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
boss = rank==0
# Seed RNG
np.random.seed(rank)
# python -m pip install mpi4py

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
training_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/training data.csv')
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/test data.csv')
valid_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/valid data.csv')
training_data = np.array(training_data)
test_data = np.array(test_data)
valid_data = np.array(valid_data)

In [8]:
class Model():
  def __init__(self, input_dim, output_dim, learning_rate, seed):

    torch.manual_seed(seed)

    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_units = 100
        # Instantiate model
    self.model = torch.nn.Sequential(
    torch.nn.Linear(self.input_dim, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.output_dim, bias=True)
    ).to(torch.device('cpu'))
    # Instantiate optimizer
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)

  def adjust_learning_rate(self):
    for param_group in self.optimizer.param_groups:
      param_group['lr'] = param_group['lr']*0.999099

  def softplus(self, x):
    """ Compute softplus """
    softplus = torch.log(1+torch.exp(x))
    # Avoid infinities due to taking the exponent
    softplus = torch.where(softplus==float('inf'), x, softplus)
    return softplus

  def NLL(self, mean, var, truth):
    diff = torch.sub(truth, mean)
    var = self.softplus(var)
      # Compute loss 
    loss = torch.mean(torch.div(diff**2, 2*var))
    loss += torch.mean(0.5*torch.log(var))
    return loss.sum()

  def forward(self, inputs, input_type):
    """ Forward pass for a given input
        :input:     (state,action)-pair
        :returns:   means and variances given input
    """
    # Compute output of model
    if input_type == "nparray":            
      x = torch.from_numpy(inputs).float()
      x = x.view(-1,self.input_dim)
      out = self.model(x)
      mean, var = torch.split(out, self.output_dim//2, dim=1)
      var = self.softplus(var)
      return mean.detach().numpy(), var.detach().numpy()
    else:
      inputs = inputs.view(-1,self.input_dim)
      out = self.model(inputs)
      mean, var = torch.split(out, self.output_dim//2, dim=1)
      var = self.softplus(var)
      return mean, var
      
  def step(self, inputs, true_out):
    """ Execute gradient step given the samples in the minibatch """
    # Convert input and true_out to useable tensors
    x = torch.from_numpy(inputs).float()
    y = torch.from_numpy(true_out).float()
      

    # Compute output of model
    out = self.model(x)
    mean, var = torch.split(out, self.output_dim//2, dim=1)
    


    # Compute loss 
    self.nll = self.NLL(mean, var, y)

    # Backpropagate the loss
    self.optimizer.zero_grad()
    self.nll.backward()
    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 10)
    self.optimizer.step()

    self.adjust_learning_rate()

  def compute_errors(self, train_in, validation_in, test_in):
    """ Compute loss on the training, validation and test data """
    # Training data
    train_in = torch.from_numpy(train_in).float()
    train_in, train_out = torch.split(train_in, [21,14], dim=1)
    mean, var = self.forward(train_in, "tensor")
    train_loss = self.NLL(mean, var, train_out).item()

    # Validation data
    validation_in = torch.from_numpy(validation_in).float()
    validation_in, val_out = torch.split(validation_in, [21,14], dim=1)
    mean, var = self.forward(validation_in, "tensor")
    val_loss = self.NLL(mean, var, val_out).item()

    # Test data
    test_in = torch.from_numpy(test_in).float()
    test_in, test_out = torch.split(test_in, [21,14], dim=1)
    mean, var = self.forward(test_in, "tensor")
    test_loss = self.NLL(mean, var, test_out).item()

    return train_loss, val_loss, test_loss

In [33]:
ensemble_size = 4          # Ensemble size per core
epochs = 20000
learning_rate = 0.000012
training_samples = 1000
validation_samples = training_samples
test_samples = 1000
batch_size = 16
measurements = epochs//200  # Measure every n steps

# Define network architecture
input_dim = 21
output_dim = 28
    # Define systems
    # current available data sets: SimpleTwoDimensional(N, seed)
datamic_systems = ["linear"]

seeds = np.random.randint(1e4, size=ensemble_size)
    # Allocate for losses
train_error = np.zeros((ensemble_size, measurements))
validation_error = np.zeros(train_error.shape)
test_error = np.zeros(train_error.shape)
    
for system in datamic_systems:
        # Instantiate class objects
        # data = datamics.datamics(training_samples, system)
        # Initialize models
  ensemble = [Model(input_dim, output_dim, learning_rate, seeds[i]) for i in range(ensemble_size)]
      # Initialize & allocate
  ensemble_mean = np.zeros((test_samples, 14))
  ensemble_var = np.zeros((test_samples, 14))

      

  # Train an ensemble of probabilistic networks:
  i = 0
  for model in ensemble:
    j = 0
    for epoch in range(epochs):
      indices = np.random.choice(range(training_samples-1), size=batch_size,replace=False)
      minibatch=training_data[indices]
      model_in = minibatch[:,:21]
      y = minibatch[:,21:]
      model.step(model_in, y)
      if (epoch+1)%(epochs//measurements)==0:
        # Compute error on train, test and validation sets
        train_error[i,j], validation_error[i,j], test_error[i,j] = model.compute_errors(training_data, valid_data, test_data)
        j += 1
      if (epoch+1)%100 == 0:
        print('Epoch %s, Train loss %s, Valid loss %s, Test loss %s'%(epoch, train_error[i,j-1],validation_error[i,j-1], test_error[i,j-1]))

      # Test model on training samples

  #mean, var = model.forward(test_data[:,:21], "nparray")


      # Add to ensemble mean and variance
  #ensemble_mean += mean 
  #ensemble_var += var + mean**2

  i += 1
    


        
   



Epoch 99, Train loss 0.0, Valid loss 0.0, Test loss 0.0
Epoch 199, Train loss 304.93646240234375, Valid loss 308.46697998046875, Test loss 314.2660217285156
Epoch 299, Train loss 304.93646240234375, Valid loss 308.46697998046875, Test loss 314.2660217285156
Epoch 399, Train loss 180.85240173339844, Valid loss 182.2140350341797, Test loss 187.1732635498047
Epoch 499, Train loss 180.85240173339844, Valid loss 182.2140350341797, Test loss 187.1732635498047
Epoch 599, Train loss 112.6772232055664, Valid loss 113.50374603271484, Test loss 116.99335479736328
Epoch 699, Train loss 112.6772232055664, Valid loss 113.50374603271484, Test loss 116.99335479736328
Epoch 799, Train loss 77.87274932861328, Valid loss 78.4239273071289, Test loss 80.82411193847656
Epoch 899, Train loss 77.87274932861328, Valid loss 78.4239273071289, Test loss 80.82411193847656
Epoch 999, Train loss 58.916873931884766, Valid loss 59.322853088378906, Test loss 61.018409729003906
Epoch 1099, Train loss 58.916873931884766,

KeyboardInterrupt: ignored

In [16]:
def simulate_system(x, u):
  ensemble_mean = np.zeros((1,14))
  ensemble_mean = torch.from_numpy(ensemble_mean)
  ensemble_var = np.zeros((1,14))
  ensemble_var = torch.from_numpy(ensemble_var)
  x = x.T
  u = u.T
  x = np.hstack((x,u))
  x = x.astype(np.float32)
  #x = torch.from_numpy(x)
  for model in ensemble:
    mean, var = model.forward(x, "nparray")
    ensemble_mean += mean
    ensemble_var += var + mean ** 2
    print(mean)
    
  #gathered_means = comm.gather(ensemble_mean)
  #gathered_vars = comm.gather(ensemble_var)
  #stacked_means = np.stack(gathered_means, axis=0)
  #stacked_vars = np.stack(gathered_vars, axis=0)
  #assert(stacked_means.shape == stacked_vars.shape)
  #ensemble_mean = stacked_means / ensemble_size
  #ensemble_var = stacked_vars / ensemble_size
  #ensemble_var = ensemble_var - ensemble_mean**2
  ensemble_mean = ensemble_mean / ensemble_size
  ensemble_var = ensemble_var / ensemble_size
  ensemble_var = ensemble_var - ensemble_mean**2

  #ensemble_mean=ensemble_mean.detach().numpy()
  #ensemble_var=ensemble_var.detach().numpy()
  #ensemble_var = ensemble_var.reshape((14,1))
    #for i in range(14):
        #if ensemble_var[i]<=0:
            #ensemble_var[i] = 0.0001
        #if ensemble_var[i] != float:
            #ensemble_var[i] = 10000
  return ensemble_mean,ensemble_var

In [19]:
ZZ = np.array(training_data[1,:14])
UU = np.array(training_data[1,14:21])
simulate_system(ZZ, UU)

[[-1.1337963  -1.2965276   0.8364221   1.1230072   0.7402898  -0.76636934
   0.04833825  4.1550684   4.049585    5.3085546   0.2869519  -3.808255
  -3.0492458  14.856499  ]]
[[ 3.4293373   4.028005    1.3922765   0.27590925 -0.24528256 -0.2998793
   0.41948146  3.3247755   4.1542635  -2.7659194   2.6210191  -9.371803
  -2.609622   -3.349002  ]]
[[-0.33334512 -0.5841472   0.19395602  2.168904    0.30806386  0.32784948
  -0.05072077  1.63087    -3.4722543  -1.9151219  -0.12373321 -9.704905
  -1.3379205   1.8444126 ]]
[[  0.20540015  -0.10438987   0.4650936   -0.80695325   0.17842488
    0.34767938  -0.15283595   7.8216257    0.80170804  -5.7370768
   -6.8539004   -3.2436802  -19.528582    10.439626  ]]


(tensor([[ 0.5419,  0.5107,  0.7219,  0.6902,  0.2454, -0.0977,  0.0661,  4.2331,
           1.3833, -1.2774, -1.0174, -6.5322, -6.6313,  5.9479]],
        dtype=torch.float64),
 tensor([[ 7.0661,  8.3468,  1.9046,  3.9016,  3.0293,  1.3511,  4.8771, 20.4033,
          22.3667, 36.0519, 27.8954, 42.5745, 82.6791, 91.6404]],
        dtype=torch.float64))

In [20]:
print(training_data[1,21:])

[-6.5633120e-01 -3.3784660e-01  7.0910764e-01  3.7938920e-01
 -1.1324311e-01  2.3932531e-02 -1.6888889e+00  6.0335493e+00
 -3.7846596e+00  2.3132986e+01  5.7166986e+00  2.8675690e+01
 -5.8162304e+01 -1.0000000e+02]


In [48]:
ZZ = np.array(training_data[0,:14])
UU = np.array(training_data[0,14:21])
simulate_system(ZZ, UU)

[[-0.48502004  0.8605413  -1.7585945   0.6993234   0.55698407  0.23327248
  -0.9355381  -4.790979    2.683198    3.048272    2.359308   -3.0158556
   7.388413   -3.2617886 ]]
[[ 0.25046027 -1.5253627  -1.5284578   0.8861569  -0.11947447  1.487737
  -1.3665055  -1.9053917   0.37650782 -0.90301704 -0.2531714   3.581868
  -6.3261127  -2.6135273 ]]
[[-0.2069063  -1.0923793  -0.98949605 -0.70573354  1.0254303  -0.45563734
  -0.33450377 -2.0688815   1.1788868  -2.5026858   0.05464757  4.252819
  -1.353702    1.3997132 ]]
[[  0.05011452   1.7463226    0.54926676  -0.30345455   0.31023115
   -0.5115269   -0.01689413  -1.648943    -0.04219985 -12.346002
    1.1288185   -9.253822     2.6356936    3.1973126 ]]


(tensor([[-9.7838e-02, -2.7195e-03, -9.3182e-01,  1.4407e-01,  4.4329e-01,
           1.8846e-01, -6.6336e-01, -2.6035e+00,  1.0491e+00, -3.1759e+00,
           8.2240e-01, -1.1087e+00,  5.8607e-01, -3.1957e-01]],
        dtype=torch.float64),
 tensor([[ 2.4228,  4.3733,  5.3372,  2.6494,  3.5205,  3.5906,  3.0167, 17.7586,
          13.3182, 47.5430, 19.4407, 56.3754, 48.6377, 40.2676]],
        dtype=torch.float64))

In [11]:
# Gather data from all cores
gathered_means = comm.gather(ensemble_mean)
gathered_vars = comm.gather(ensemble_var)
gathered_train_err = comm.gather(train_error)
gathered_val_err = comm.gather(validation_error)
gathered_test_err = comm.gather(test_error)
# Compute ensemble mean and averages
stacked_means = np.stack(gathered_means, axis=0)

stacked_vars = np.stack(gathered_vars, axis=0)
assert(stacked_means.shape == stacked_vars.shape)
ensemble_mean = stacked_means / ensemble_size
ensemble_var = stacked_vars / ensemble_size
ensemble_var = ensemble_var - ensemble_mean**2
print(ensemble_mean)

# Save ensemble
#np.save("data/ensemble_mean_%s"%(system), ensemble_mean)
#np.save("data/ensemble_var_%s"%(system), ensemble_var)
# Compute mean errors
stacked_train_err = np.stack(gathered_train_err, axis=0)
mean_train_err = np.mean(np.mean(gathered_train_err, axis=0), axis=0)
#np.save("data/train_error_%s"%(system), mean_train_err)
stacked_val_err = np.stack(gathered_val_err, axis=0)
mean_val_err = np.mean(np.mean(gathered_val_err, axis=0), axis=0)
#np.save("data/val_error_%s"%(system), mean_val_err)
stacked_test_err = np.stack(gathered_test_err, axis=0)
mean_test_err = np.mean(np.mean(gathered_test_err, axis=0), axis=0)
#np.save("data/test_error_%s"%(system), mean_test_err)


# Save things for plotting
#np.save("data/test_samples", data.test_data)
#print("\n...")

[[[ 2.14819041 -0.62289528 -0.2587198  ... -2.04153483 -2.85288429
   -4.69466043]
  [ 3.16661927 -0.16044442 -0.31342744 ... -4.06609368 -2.9272452
   -1.51780862]
  [ 1.75710507 -0.48084529  0.17528443 ... -2.75613517 -2.20122683
   -0.20081882]
  ...
  [ 1.68377027 -0.84343077  0.10162868 ... -3.37603571 -3.5194281
   -0.73134935]
  [ 2.62247431 -0.0666911  -1.13640465 ... -3.08553943 -3.25247735
   -1.29030269]
  [ 2.36711986 -0.5144839   0.20863129 ... -3.22959562 -2.44806156
   -0.5163473 ]]]
